In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

asset_list = [
    "SPY", "QQQ", "DIA",
    "EWJ", "EEM", "VGK",
    "TLT", "IEF", "BND",
    "GLD", "SLV", "USO",
    "EURUSD=X", "JPY=X", "AUDUSD=X",
    "BTC-USD", "ETH-USD"
]
MACD_FAST = 12
MACD_SLOW = 26
MACD_SIGNAL = 9

def get_data(ticker):
    df = yf.download(ticker, interval="1d", start="2000-01-01", end="2024-01-01")
    df = df[['Close']].dropna()
    return df

def apply_strategy(df, fast=MACD_FAST, slow=MACD_SLOW, signal=MACD_SIGNAL):
    df = df.copy()
    df[f'{fast}_EMA'] = df['Close'].ewm(span=fast, adjust=False).mean()
    df[f'{slow}_EMA'] = df['Close'].ewm(span=slow, adjust=False).mean()
    df['MACD'] = df[f'{fast}_EMA'] - df[f'{slow}_EMA']
    df['MACD_Signal'] = df['MACD'].ewm(span=signal, adjust=False).mean()
    df['MACD_Hist'] = df['MACD'] - df['MACD_Signal']
    
    df['signal'] = 0
    df['MACD_Strategy'] = np.where(df['MACD'] > df['MACD_Signal'], 1, -1)   # short
    df['signal'] = df['MACD_Strategy']
    df['signal'] = df['signal'].shift(1)                         # trade next day
    df['signal'].fillna(0, inplace=True)

    # daily returns
    df['ret'] = df['Close'].pct_change().fillna(0)

    # strategy returns
    df['strategy_ret'] = df['signal'] * df['ret']

    return df[['signal','strategy_ret', 'ret']]


def walk_forward(df, train_years=3, test_years=1):

    df = df.copy()
    results = []

    start = df.index.min()

    while True:
        train_start = start
        train_end = train_start + pd.DateOffset(years=train_years)
        test_end = train_end + pd.DateOffset(years=test_years)

        # break if test window exceeds data
        if test_end > df.index.max():
            break

        # slice windows
        train = df[(df.index >= train_start) & (df.index < train_end)]
        test = df[(df.index >= train_end) & (df.index < test_end)]

        # warm-up buffer for MACD (100 days)
        warmup = df[(df.index >= train_start - pd.DateOffset(days=200)) &
                    (df.index < train_start)]

        # train + test combined for MA calculation
        combined = pd.concat([warmup, train, test])

        # apply strategy
        window_results = apply_strategy(combined)

        # keep only test window results
        window_results = window_results.loc[test.index]

        results.append(window_results)

        # move forward
        start = start + pd.DateOffset(years=test_years)

    final = pd.concat(results)
    return final

ticker = "SPY"
df = get_data(ticker)

results = walk_forward(df, train_years=3, test_years=1)

# cumulative returns
strategy_curve = (1 + results['strategy_ret']).cumprod()
buyhold_curve = (1 + results['ret']).cumprod()

plt.figure(figsize=(13,6))
plt.plot(strategy_curve, label="Strategy")
plt.plot(buyhold_curve, label="Buy & Hold")
plt.title("Walk-Forward MACD Performance")
plt.legend()
plt.show()


In [ ]:
def run_strategy_on_asset(ticker):
    df = get_data(ticker)
    results = walk_forward(df)
    return results.dropna()

In [ ]:
from fredapi import Fred

fred_key = '(your fred api key)'
fred = Fred(api_key=fred_key)


In [ ]:
RISK_FREE_RATE = fred.get_series(series_id='TB3MS', observation_start = '2000-01-01', observation_end='2024-01-01').iloc[-1]

def compute_performance_metrics(df, risk_free_rate=RISK_FREE_RATE):
    df = df.copy()
    
    # Make sure index is datetime
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)
    
    init_equity = 10000
    
    # Correct equity curve calculation
    df['equity'] = init_equity * (1 + df['strategy_ret']).cumprod()
    
    # 1. CAGR - Use actual date range from the results
    total_years = (df.index[-1] - df.index[0]).days / 365.25
    final_equity = df['equity'].iloc[-1]
    cagr = (final_equity / init_equity) ** (1 / total_years) - 1
    
    # 2. Max Drawdown
    roll_max = df['equity'].cummax()
    drawdown = (df['equity'] - roll_max) / roll_max
    max_drawdown = drawdown.min()
    
    # 3. Sharpe Ratio (annualized)
    daily_rf = risk_free_rate / 252
    excess_return = df['strategy_ret'] - daily_rf
    sharpe = np.sqrt(252) * excess_return.mean() / excess_return.std() if excess_return.std() != 0 else np.nan
    
    # 4. Sortino Ratio (only downside volatility)
    downside = excess_return[excess_return < 0]
    sortino = np.sqrt(252) * excess_return.mean() / downside.std() if len(downside) > 0 and downside.std() != 0 else np.nan
    
    # 5. Profit Factor
    gross_profit = df['strategy_ret'][df['strategy_ret'] > 0].sum()
    gross_loss = abs(df['strategy_ret'][df['strategy_ret'] < 0].sum())
    profit_factor = gross_profit / gross_loss if gross_loss != 0 else np.nan
    
    # 6. Rolling Sharpe (30-day window)
    rolling_excess = df['strategy_ret'].rolling(30).mean() - (risk_free_rate / 252)
    df['Rolling_Sharpe'] = (rolling_excess / df['strategy_ret'].rolling(30).std()) * np.sqrt(252)
    
    metrics = {
        'CAGR': cagr,
        'Max Drawdown': max_drawdown,
        'Sharpe Ratio': sharpe,
        'Sortino Ratio': sortino,
        'Profit Factor': profit_factor,
        '30 days Rolling Sharpe': df['Rolling_Sharpe'].iloc[-30:].mean()
    }
    
    return metrics, df

def compute_jensens_alpha(df, risk_free_rate=RISK_FREE_RATE):
    rp = (1 + df['strategy_ret'].mean())**252 - 1
    rm = (1 + df['ret'].mean())**252 - 1
    cov = np.cov(df['strategy_ret'].dropna(), df['ret'].dropna())[0,1]
    var = np.var(df['ret'].dropna())
    beta = cov / var
    alpha = rp - (risk_free_rate + beta * (rm - risk_free_rate))

    return alpha

for ticker in asset_list:
    results = run_strategy_on_asset(ticker)
    jensens_alpha = compute_jensens_alpha(results)
    metrics, result_df = compute_performance_metrics(results)
    print(f'{ticker}')
    for k, v in metrics.items():
        print(f"{k}: {v:.4f}")

In [ ]:
RISK_FREE_RATE = fred.get_series(series_id='TB3MS', observation_start = '2000-01-01', observation_end='2024-01-01').iloc[-1]

def compute_performance_metrics(df, risk_free_rate=RISK_FREE_RATE):
    df = df.copy()
    
    # Make sure index is datetime
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)
    
    init_equity = 10000
    
    # Correct equity curve calculation
    df['equity'] = init_equity * (1 + df['strategy_ret']).cumprod()
    
    # 1. CAGR - Use actual date range from the results
    total_years = (df.index[-1] - df.index[0]).days / 365.25
    final_equity = df['equity'].iloc[-1]
    cagr = (final_equity / init_equity) ** (1 / total_years) - 1
    
    # 2. Max Drawdown
    roll_max = df['equity'].cummax()
    drawdown = (df['equity'] - roll_max) / roll_max
    max_drawdown = drawdown.min()
    
    # 3. Sharpe Ratio (annualized)
    daily_rf = risk_free_rate / 252
    excess_return = df['strategy_ret'] - daily_rf
    sharpe = np.sqrt(252) * excess_return.mean() / excess_return.std() if excess_return.std() != 0 else np.nan
    
    # 4. Sortino Ratio (only downside volatility)
    downside = excess_return[excess_return < 0]
    sortino = np.sqrt(252) * excess_return.mean() / downside.std() if len(downside) > 0 and downside.std() != 0 else np.nan
    
    # 5. Profit Factor
    gross_profit = df['strategy_ret'][df['strategy_ret'] > 0].sum()
    gross_loss = abs(df['strategy_ret'][df['strategy_ret'] < 0].sum())
    profit_factor = gross_profit / gross_loss if gross_loss != 0 else np.nan
    
    # 6. Rolling Sharpe (30-day window)
    rolling_excess = df['strategy_ret'].rolling(30).mean() - (risk_free_rate / 252)
    df['Rolling_Sharpe'] = (rolling_excess / df['strategy_ret'].rolling(30).std()) * np.sqrt(252)
    
    metrics = {
        'CAGR': cagr,
        'Max Drawdown': max_drawdown,
        'Sharpe Ratio': sharpe,
        'Sortino Ratio': sortino,
        'Profit Factor': profit_factor,
        '30 days Rolling Sharpe': df['Rolling_Sharpe'].iloc[-30:].mean()
    }
    
    return metrics, df

def compute_jensens_alpha(df, risk_free_rate=RISK_FREE_RATE):
    rp = (1 + df['strategy_ret'].mean())**252 - 1
    rm = (1 + df['ret'].mean())**252 - 1
    cov = np.cov(df['strategy_ret'].dropna(), df['ret'].dropna())[0,1]
    var = np.var(df['ret'].dropna())
    beta = cov / var
    alpha = rp - (risk_free_rate + beta * (rm - risk_free_rate))

    return alpha

for ticker in asset_list:
    results = run_strategy_on_asset(ticker)
    jensens_alpha = compute_jensens_alpha(results)
    metrics, result_df = compute_performance_metrics(results)
    print(f'{ticker}')
    for k, v in metrics.items():
        print(f"{k}: {v:.4f}")

In [ ]:
from scipy.stats import ttest_rel, wilcoxon, binomtest
import numpy as np

def statistical_tests(res):
    strat = res['strategy_ret']
    bench = res['ret']
    diff = strat - bench
    
    tests = {}

    # Paired t-test
    t_stat, p_t = ttest_rel(strat, bench)
    tests['paired_t_p'] = p_t

    # Wilcoxon test (non-parametric)
    w_stat, p_w = wilcoxon(strat, bench)
    tests['wilcoxon_p'] = p_w

    # Directional accuracy
    future = res['ret'].shift(-1)
    correct = ((res['signal']==1)&(future>0)) | ((res['signal']==0)&(future<0))
    accuracy = correct.mean()
    tests['accuracy'] = accuracy

    # Binomial test for accuracy vs 0.5
    tests['accuracy_binom_p'] = binomtest(
        correct.sum(), n=len(correct), p=0.5
    )

    return tests

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

def granger_test(res):
    df = res[['ret', 'signal']].dropna()
    # Does signal cause returns?
    g1 = grangercausalitytests(df[['ret','signal']], maxlag=5, verbose=False)
    # Does return cause signal? (control test)
    g2 = grangercausalitytests(df[['signal','ret']], maxlag=5, verbose=False)
    return g1, g2

In [ ]:
def random_permutation_test(res, n=5000):
    real_perf = (1 + res['strategy_ret']).prod()

    random_perf = []
    for _ in range(n):
        shuffled = res['ret'].sample(frac=1, replace=False).values
        shuffled_strat = res['signal'].values * shuffled
        random_perf.append((1 + shuffled_strat).prod())

    random_perf = np.array(random_perf)
    p_value = np.mean(random_perf > real_perf)

    return real_perf, random_perf.mean(), p_value

In [ ]:
granger_test_results = {}

for ticker in asset_list:
    results = run_strategy_on_asset(ticker)
    g1, g2 = granger_test(results)
    granger_test_results[ticker] = {
        'Does signal cause returns?' : g1,
        'Does return cause signal? (control test)' : g2
    }

In [ ]:
granger_test_results

In [ ]:
final_results = {}

for ticker in asset_list:
    res = run_strategy_on_asset(ticker)
    tests = statistical_tests(res)
    g1, g2 = granger_test(res)
    perm_real, perm_mean, perm_p = random_permutation_test(res)

    final_results[ticker] = {
        **tests,
        'shuffle_mean_perf': perm_mean,
        'shuffle_p_value': perm_p,
    }


In [ ]:
final_results

In [ ]:
def seperate_by_regimes(results, regimes):
    common_dates = results.index.intersection(regimes.index)
    
    if len(common_dates) == 0:
        print("WARNING: No overlapping dates between results and regimes!")
        return results.copy(), pd.DataFrame()
    
    # Filter regimes untuk tanggal yang ada di results
    aligned_regimes = regimes.loc[common_dates]
    
    # Separate
    low_vol_dates = aligned_regimes[aligned_regimes == 0].index
    high_vol_dates = aligned_regimes[aligned_regimes == 1].index
    
    low_vol = results.loc[low_vol_dates].copy()
    high_vol = results.loc[high_vol_dates].copy()
    
    print(f"\nFound {len(low_vol)} low volatility days")
    print(f"Found {len(high_vol)} high volatility days")
    print(f"Total: {len(low_vol) + len(high_vol)} days")
    
    return low_vol, high_vol

In [ ]:
def analyze_by_regime(low_vol, high_vol):
    print("\n" + "="*70)
    print("REGIME ANALYSIS - WALK-FORWARD RESULTS")
    print("="*70)
    
    total_days = len(low_vol) + len(high_vol)
    print(f"\nLow Volatility Days:  {len(low_vol):>6} ({len(low_vol)/total_days*100:>5.1f}%)")
    print(f"High Volatility Days: {len(high_vol):>6} ({len(high_vol)/total_days*100:>5.1f}%)")
    
    print("\n" + "-"*70)
    print(f"{'Metric':<35} {'Low Vol':>15} {'High Vol':>15}")
    print("-"*70)
    
    # Low Vol Stats
    if len(low_vol) > 0:
        low_strat_ret = low_vol['strategy_ret'].mean() * 252 * 100
        low_strat_vol = low_vol['strategy_ret'].std() * np.sqrt(252) * 100
        low_strat_sharpe = low_vol['strategy_ret'].mean() / low_vol['strategy_ret'].std() * np.sqrt(252) if low_vol['strategy_ret'].std() > 0 else 0
        low_bh_ret = low_vol['ret'].mean() * 252 * 100
        low_bh_vol = low_vol['ret'].std() * np.sqrt(252) * 100
        low_bh_sharpe = low_vol['ret'].mean() / low_vol['ret'].std() * np.sqrt(252) if low_vol['ret'].std() > 0 else 0
        low_win = (low_vol['strategy_ret'] > 0).sum() / len(low_vol) * 100
        low_strat_total = (1 + low_vol['strategy_ret']).prod() - 1
        low_bh_total = (1 + low_vol['ret']).prod() - 1
    else:
        low_strat_ret = low_strat_vol = low_strat_sharpe = 0
        low_bh_ret = low_bh_vol = low_bh_sharpe = low_win = 0
        low_strat_total = low_bh_total = 0
    
    # High Vol Stats
    if len(high_vol) > 0:
        high_strat_ret = high_vol['strategy_ret'].mean() * 252 * 100
        high_strat_vol = high_vol['strategy_ret'].std() * np.sqrt(252) * 100
        high_strat_sharpe = high_vol['strategy_ret'].mean() / high_vol['strategy_ret'].std() * np.sqrt(252) if high_vol['strategy_ret'].std() > 0 else 0
        high_bh_ret = high_vol['ret'].mean() * 252 * 100
        high_bh_vol = high_vol['ret'].std() * np.sqrt(252) * 100
        high_bh_sharpe = high_vol['ret'].mean() / high_vol['ret'].std() * np.sqrt(252) if high_vol['ret'].std() > 0 else 0
        high_win = (high_vol['strategy_ret'] > 0).sum() / len(high_vol) * 100
        high_strat_total = (1 + high_vol['strategy_ret']).prod() - 1
        high_bh_total = (1 + high_vol['ret']).prod() - 1
    else:
        high_strat_ret = high_strat_vol = high_strat_sharpe = 0
        high_bh_ret = high_bh_vol = high_bh_sharpe = high_win = 0
        high_strat_total = high_bh_total = 0
    
    print(f"{'Total Return (Strategy)':<35} {low_strat_total*100:>14.2f}% {high_strat_total*100:>14.2f}%")
    print(f"{'Total Return (Buy&Hold)':<35} {low_bh_total*100:>14.2f}% {high_bh_total*100:>14.2f}%")
    print(f"{'Ann. Return (Strategy)':<35} {low_strat_ret:>14.2f}% {high_strat_ret:>14.2f}%")
    print(f"{'Ann. Return (Buy&Hold)':<35} {low_bh_ret:>14.2f}% {high_bh_ret:>14.2f}%")
    print(f"{'Ann. Volatility (Strategy)':<35} {low_strat_vol:>14.2f}% {high_strat_vol:>14.2f}%")
    print(f"{'Ann. Volatility (Buy&Hold)':<35} {low_bh_vol:>14.2f}% {high_bh_vol:>14.2f}%")
    print(f"{'Sharpe Ratio (Strategy)':<35} {low_strat_sharpe:>15.2f} {high_strat_sharpe:>15.2f}")
    print(f"{'Sharpe Ratio (Buy&Hold)':<35} {low_bh_sharpe:>15.2f} {high_bh_sharpe:>15.2f}")
    print(f"{'Win Rate (Strategy)':<35} {low_win:>14.2f}% {high_win:>14.2f}%")
    
    print("="*70)

In [ ]:
def get_daily_fred_series(series_id, results_index, start='2000-01-01', end='2024-01-01', plot=False):

    # Download series
    series = fred.get_series(series_id=series_id, observation_start=start, observation_end=end)

    # Optional plot
    if plot:
        plt.figure(figsize=(10,4))
        plt.plot(series)
        plt.title(f"FRED Series: {series_id}")
        plt.grid(True)
        plt.show()

    # Convert to DataFrame
    df = series.to_frame(name=series_id)
    df.index = pd.to_datetime(df.index)

    # Resample to daily and forward-fill
    df_daily = df.resample("D").ffill()
    df_aligned = df_daily.reindex(results_index).ffill()

    return df_aligned

def label_recessions(df, series_id):
    df = df.copy()
    df["recession"] = 0
    
    in_recession = False
    
    for i in range(len(df)):
        value = df[series_id].iat[i]
        
        if not in_recession:
            # recession starts
            if value > 67:
                in_recession = True
                df["recession"].iat[i] = 1
        else:
            # currently in recession
            df["recession"].iat[i] = 1
            
            # recession ends
            if value < 33:
                in_recession = False
    
    return df

In [ ]:
def seperate_by_recession(results, recession):
    common_dates = results.index.intersection(recession.index)
    
    if len(common_dates) == 0:
        print("WARNING: No overlapping dates between results and recession!")
        return results.copy(), pd.DataFrame()
    
    # Filter regimes untuk tanggal yang ada di results
    recession_regimes = recession.loc[common_dates]
    
    # Separate
    non_recession_dates_idx = recession_regimes[recession_regimes == 0].index
    recession_dates_idx = recession_regimes[recession_regimes == 1].index
    
    non_recession_dates_df = results.loc[non_recession_dates_idx].copy()
    recession_dates_df= results.loc[recession_dates_idx].copy()
    
    print(f"\nFound {len(non_recession_dates_df)} non recession days")
    print(f"Found {len(recession_dates_df)} recession days")
    print(f"Total: {len(non_recession_dates_df) + len(recession_dates_df)} days")
    
    return recession_dates_df, non_recession_dates_df

In [ ]:
def analyze_by_recession(recession_dates, non_recession_dates):
    print("\n" + "="*70)
    print("REGIME ANALYSIS - WALK-FORWARD RESULTS")
    print("="*70)
    
    total_days = len(non_recession_dates) + len(recession_dates)
    print(f"\nNon-Recession Days:  {len(non_recession_dates):>6} ({len(non_recession_dates)/total_days*100:>5.1f}%)")
    print(f"Recession Days: {len(recession_dates):>6} ({len(recession_dates)/total_days*100:>5.1f}%)")
    
    print("\n" + "-"*70)
    print(f"{'Metric':<35} {'Non-Recession':>15} {'Recession':>15}")
    print("-"*70)
    
    # Low Vol Stats
    if len(non_recession_dates) > 0:
        low_strat_ret = non_recession_dates['strategy_ret'].mean() * 252 * 100
        low_strat_vol = non_recession_dates['strategy_ret'].std() * np.sqrt(252) * 100
        low_strat_sharpe = non_recession_dates['strategy_ret'].mean() / non_recession_dates['strategy_ret'].std() * np.sqrt(252) if non_recession_dates['strategy_ret'].std() > 0 else 0
        low_bh_ret = non_recession_dates['ret'].mean() * 252 * 100
        low_bh_vol = non_recession_dates['ret'].std() * np.sqrt(252) * 100
        low_bh_sharpe = non_recession_dates['ret'].mean() / non_recession_dates['ret'].std() * np.sqrt(252) if non_recession_dates['ret'].std() > 0 else 0
        low_win = (non_recession_dates['strategy_ret'] > 0).sum() / len(non_recession_dates) * 100
        low_strat_total = (1 + non_recession_dates['strategy_ret']).prod() - 1
        low_bh_total = (1 + non_recession_dates['ret']).prod() - 1
    else:
        low_strat_ret = low_strat_vol = low_strat_sharpe = 0
        low_bh_ret = low_bh_vol = low_bh_sharpe = low_win = 0
        low_strat_total = low_bh_total = 0
    
    # High Vol Stats
    if len(recession_dates) > 0:
        high_strat_ret = recession_dates['strategy_ret'].mean() * 252 * 100
        high_strat_vol = recession_dates['strategy_ret'].std() * np.sqrt(252) * 100
        high_strat_sharpe = recession_dates['strategy_ret'].mean() / recession_dates['strategy_ret'].std() * np.sqrt(252) if recession_dates['strategy_ret'].std() > 0 else 0
        high_bh_ret = recession_dates['ret'].mean() * 252 * 100
        high_bh_vol = recession_dates['ret'].std() * np.sqrt(252) * 100
        high_bh_sharpe = recession_dates['ret'].mean() / recession_dates['ret'].std() * np.sqrt(252) if recession_dates['ret'].std() > 0 else 0
        high_win = (recession_dates['strategy_ret'] > 0).sum() / len(recession_dates) * 100
        high_strat_total = (1 + recession_dates['strategy_ret']).prod() - 1
        high_bh_total = (1 + recession_dates['ret']).prod() - 1
    else:
        high_strat_ret = high_strat_vol = high_strat_sharpe = 0
        high_bh_ret = high_bh_vol = high_bh_sharpe = high_win = 0
        high_strat_total = high_bh_total = 0
    
    print(f"{'Total Return (Strategy)':<35} {low_strat_total*100:>14.2f}% {high_strat_total*100:>14.2f}%")
    print(f"{'Total Return (Buy&Hold)':<35} {low_bh_total*100:>14.2f}% {high_bh_total*100:>14.2f}%")
    print(f"{'Ann. Return (Strategy)':<35} {low_strat_ret:>14.2f}% {high_strat_ret:>14.2f}%")
    print(f"{'Ann. Return (Buy&Hold)':<35} {low_bh_ret:>14.2f}% {high_bh_ret:>14.2f}%")
    print(f"{'Ann. Volatility (Strategy)':<35} {low_strat_vol:>14.2f}% {high_strat_vol:>14.2f}%")
    print(f"{'Ann. Volatility (Buy&Hold)':<35} {low_bh_vol:>14.2f}% {high_bh_vol:>14.2f}%")
    print(f"{'Sharpe Ratio (Strategy)':<35} {low_strat_sharpe:>15.2f} {high_strat_sharpe:>15.2f}")
    print(f"{'Sharpe Ratio (Buy&Hold)':<35} {low_bh_sharpe:>15.2f} {high_bh_sharpe:>15.2f}")
    print(f"{'Win Rate (Strategy)':<35} {low_win:>14.2f}% {high_win:>14.2f}%")
    
    print("="*70)

    

In [ ]:
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression

models = {}
all_results = {}

for ticker in asset_list:
    results = run_strategy_on_asset(ticker)
    all_results[ticker] = results
    market_ret = results['ret']
    # Fit on in-sample window
    model = MarkovRegression(
        market_ret,
        k_regimes=2,
        trend="c",
        switching_variance=True
    )

    models[ticker] = model.fit()
    


In [ ]:
all_results['SPY']

In [ ]:
for ticker in asset_list:
    print(f"\n{'='*70}")
    print(f"ASSET: {ticker}")
    print(f"{'='*70}")
    
    # Separate and analyze
    results = all_results[ticker]
    fred_series = get_daily_fred_series('JHGDPBRINDX', results.index)
    fred_rec = label_recessions(fred_series, 'JHGDPBRINDX')
    recession_dates,  non_recession_dates = seperate_by_recession(snp, fred_rec['recession'])
    analyze_by_recession(recession_dates, non_recession_dates)

In [ ]:
for ticker in asset_list:
    print(f"\n{'='*70}")
    print(f"ASSET: {ticker}")
    print(f"{'='*70}")
    
    # Get regime from that asset's model
    regime = models[ticker].smoothed_marginal_probabilities.idxmax(axis=1)
    
    # Separate and analyze
    results = all_results[ticker]
    low_vol, high_vol = seperate_by_regimes(results, regime)
    analyze_by_regime(low_vol, high_vol)

In [ ]:
import statsmodels.api as sm

def newey_west_alpha_test(res):
    y = res['strategy_ret'] - res['ret']      # strategy excess over market
    X = np.ones(len(y))                       # constant only
    model = sm.OLS(y, X)
    nw = model.fit(cov_type='HAC', cov_kwds={'maxlags':5})
    return nw.tvalues[0], nw.pvalues[0]

for ticker in asset_list:
    results = all_results[ticker]
    t_values, p_values = newey_west_alpha_test(results)
    print(f"{ticker} t_values = {t_values}")
    print(f"{ticker} p_values = {p_values}")
    print("=" * 70)

In [ ]:
from statsmodels.tsa.stattools import adfuller, kpss

def stationarity_tests(series):
    return {
        'adf_p': adfuller(series.dropna())[1],
        'kpss_p': kpss(series.dropna(), nlags='auto')[1]
    }

for ticker in asset_list:
    results = all_results[ticker]
    adf_p, kpps_p = newey_west_alpha_test(results)
    print(f"{ticker} adf p_value = {adf_p}")
    print(f"{ticker} kpps p_value = {kpps_p}")
    print("=" * 70)

In [ ]:
import numpy as np
from scipy.stats import norm

def ledoit_wolf_sharpe_test(r1, r2):
    n = len(r1)
    mean1, mean2 = np.mean(r1), np.mean(r2)
    std1, std2  = np.std(r1, ddof=1), np.std(r2, ddof=1)

    sharpe1, sharpe2 = mean1/std1, mean2/std2
    diff = sharpe1 - sharpe2

    # LW correction components
    term1 = (mean1**2 / (2 * std1**4)) * np.var((r1 - mean1)**2)
    term2 = (mean2**2 / (2 * std2**4)) * np.var((r2 - mean2)**2)
    var_diff = term1 + term2

    z = diff / np.sqrt(var_diff / n)
    p = 1 - norm.cdf(z)  # one-sided test: is Sharpe1 > Sharpe2?

    return sharpe1, sharpe2, diff, z, p

for ticker in asset_list:
    results = all_results[ticker]
    s1, s2, diff, z, p = ledoit_wolf_sharpe_test(results['strategy_ret'], results['ret'])
    print(f'p value for {ticker} = {p}')

In [ ]:
import numpy as np

def bootstrap_significance(strategy_returns, n_boot=5000):
    actual_mean = np.mean(strategy_returns)

    boot_means = []
    for _ in range(n_boot):
        random_sign = np.random.choice([-1, 1], size=len(strategy_returns))
        boot_means.append(np.mean(strategy_returns * random_sign))

    p_value = np.mean(np.array(boot_means) >= actual_mean)
    return actual_mean, np.mean(boot_means), p_value

for ticker in asset_list:
    results = all_results[ticker]
    actual_mean, boot_mean, p_value = bootstrap_significance(results['strategy_ret'])
    print(f'{ticker} p_value = {p_value}')
